In [1]:
#import sys
#!{sys.executable} -m pip install statsmodels

In [2]:
import function
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix
import seaborn as sns
import plotly.graph_objects as go


from sklearn.preprocessing import LabelEncoder, Normalizer, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import skew, kurtosis, t, f

from ipywidgets import *
from importlib import reload


In [3]:
df = pd.read_csv("soMuchWorkIHadToDrinkOneBeer.csv" )
Y = df["review_overall"]
print(df.shape)
print(df.columns)
dfcorr = df.drop(["Name", "Beer Name (Full)","Description"], axis=1)


(3197, 25)
Index(['Name', 'Style', 'Brewery', 'Beer Name (Full)', 'Description', 'ABV',
       'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter',
       'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty',
       'review_aroma', 'review_appearance', 'review_palate', 'review_taste',
       'review_overall', 'number_of_reviews'],
      dtype='object')


## Descriptive analysis

### Mean, std, kurtosis, skewness

In [4]:
def describe_columns(dataset):
    """
    Return the dataset containing the mean, standard deviation, skewness, and kurtosis of each column in the dataset.
    """
    results = pd.DataFrame(columns=["name",'mean', 'std', 'skewness', 'kurtosis'])
    
    # iterate over each column in the dataset
    for col in dataset.columns:

        if (dataset[col].dtypes == float or dataset[col].dtypes == np.int64):
            # calculate the mean, standard deviation, skewness, and kurtosis
            mean = dataset[col].mean()
            std = dataset[col].std()
            skewness = skew(dataset[col])
            kurt = kurtosis(dataset[col])
            results.loc[col] = [ col, mean, std, skewness, kurt]

        
    return results

In [5]:
# Display table
table = describe_columns(df)
markdown_table = table.to_markdown(index=False)
print(markdown_table)

| name              |      mean |        std |   skewness |   kurtosis |
|:------------------|----------:|-----------:|-----------:|-----------:|
| ABV               |   6.52669 |   2.547    |   3.67993  |  55.2444   |
| Min IBU           |  21.1805  |  13.2422   |   1.09326  |   1.32465  |
| Max IBU           |  38.9869  |  21.3553   |   0.99054  |   0.977638 |
| Astringency       |  16.5158  |  10.4107   |   1.31567  |   2.7722   |
| Body              |  46.1295  |  25.9478   |   1.14028  |   1.61526  |
| Alcohol           |  17.056   |  17.3313   |   2.23306  |   6.28596  |
| Bitter            |  36.3644  |  25.7912   |   1.00606  |   0.938633 |
| Sweet             |  58.2709  |  34.2813   |   0.905323 |   1.31787  |
| Sour              |  33.1454  |  35.7802   |   2.52257  |   7.85566  |
| Salty             |   1.0172  |   2.13265  |   7.53925  | 115.175    |
| Fruits            |  38.5296  |  32.2966   |   0.944666 |   0.220681 |
| Hoppy             |  40.9246  |  30.4036   |   1.

### Correlation matrix

In [6]:
print(dfcorr.shape)
corr = dfcorr.corr()
fig = px.imshow(corr, title="Correlation Heat Map")
fig.show()

(3197, 22)


### Boxplot

In [7]:
function.InteractiveBoxPlot(dfcorr)

    'data': [{'type': 'box',
              'uid': 'cf098497-f910-482d-a413-e65f9…

### Nonlinearity

In [ ]:
# Display scatter plot based on the variables and the response variable
def display_plot(x, y):
    plt.scatter(x, y)
    plt.xlabel(x.name)
    plt.ylabel(y.name)
    plt.xlim(x.min(), x.max())
    plt.show()

for col in dfcorr.columns:
    if (col != "Style" and col != "Brewery" and col != "review_overall"):
        display_plot(dfcorr[col], Y)

### Standardization

In [9]:
df_quant = dfcorr.select_dtypes(include=['float64', 'int64'])
df_qual = dfcorr.select_dtypes(include=['object'])

df_standarized = (df_quant - df_quant.mean()) / df_quant.std()
dfcorr_standarized = pd.concat([df_standarized, df_qual], axis=1)


### Dummies

In [10]:
# Number of styles and breweries
nbr_style = df['Style'].nunique()
nbr_brewery = df['Brewery'].nunique()


In [11]:
print(dfcorr_standarized.shape)

# Takes the 10 most frequent breweries
most_frequents = dfcorr_standarized['Brewery'].value_counts().index[:10]

dummies = pd.DataFrame(np.zeros((len(dfcorr_standarized), len(most_frequents))), columns=most_frequents)

# Set the dummy variables to 1 if the brewery is in the most frequent breweries
for i, brewery in enumerate(dfcorr_standarized['Brewery'].values):
    if brewery in most_frequents:
        dummies.loc[i, brewery] = 1

dfcorr_standarized = pd.concat([dfcorr_standarized, dummies], axis=1)
dfcorr_standarized = dfcorr_standarized.drop('Brewery', axis=1)
dfcorr_standarized = pd.get_dummies(dfcorr_standarized, columns=['Style'], drop_first=True,prefix='Style', prefix_sep='_')



(3197, 22)


In [12]:
print(dfcorr_standarized.columns)

Index(['ABV', 'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter',
       'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty',
       'review_aroma', 'review_appearance', 'review_palate', 'review_taste',
       'review_overall', 'number_of_reviews',
       'Boston Beer Company (Samuel Adams)', 'Dogfish Head Brewery',
       'Anheuser-Busch', 'Three Floyds Brewing Co. & Brewpub',
       'Victory Brewing Company', 'Rogue Ales', 'Matt Brewing Company',
       'Short's Brewing Company', 'Great Divide Brewing Company',
       'Russian River Brewing Company', 'Style_Bock', 'Style_Brett',
       'Style_Brown Ale', 'Style_Brut', 'Style_IPA', 'Style_Lambic',
       'Style_Sour', 'Style_Trappist beer', 'Style_dark', 'Style_lager'],
      dtype='object')


## Split the dataset

In [13]:
dfcorr_standarized = dfcorr_standarized.drop("review_overall", axis=1)
X_train, X_test, y_train, y_test = train_test_split(dfcorr_standarized, Y, test_size=0.2, random_state=42)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


## OLS model

### Lasso (model selection)

In [14]:
def get_col_lasso(X_train, y_train):
    """Return the columns to delete from the dataset based on the Lasso model"""
    # Create the Lasso model
    lasso_model = LassoCV(cv=5)

    # Fit the model to the data
    lasso_model.fit(X_train, y_train)
    #print("alpha = "+str(lasso_model.alpha_))

    column_del = lasso_model.coef_ == 0
    column_del[0] = False

    return X_train.columns[column_del]

col_to_del = get_col_lasso(X_train, y_train)

print(col_to_del)

Index(['Salty', 'Boston Beer Company (Samuel Adams)',
       'Three Floyds Brewing Co. & Brewpub', 'Victory Brewing Company',
       'Rogue Ales', 'Matt Brewing Company', 'Great Divide Brewing Company',
       'Russian River Brewing Company', 'Style_Brett', 'Style_Brut',
       'Style_Lambic', 'Style_Sour', 'Style_Trappist beer'],
      dtype='object')


In [15]:
X_train_corr = X_train.drop(col_to_del, axis=1)
X_test_corr = X_test.drop(col_to_del, axis=1)

In [16]:
#Train OLS
model = sm.OLS(y_train,X_train_corr).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         review_overall   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     1040.
Date:                Sun, 08 Jan 2023   Prob (F-statistic):               0.00
Time:                        00:05:48   Log-Likelihood:                 1626.6
No. Observations:                2557   AIC:                            -3199.
Df Residuals:                    2530   BIC:                            -3041.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

### Testing hypotheses

#### Outliers

In [ ]:
def get_X_outliers(X_train):
    """Return the indexes of the outliers in the dataset with respect to X and the leverage of each entry"""

    X=X_train.values
    # Calculate X'X
    X_transpose_X = np.dot(X.T, X)

    # Calculate the inverse of X'X
    X_transpose_X_inv = np.linalg.inv(X_transpose_X)

    # Calculate the projection matrix H
    H = np.dot(X, X_transpose_X_inv)
    H = np.dot(H , X.T)

    # Calculate the leverage for each point
    H_ii = np.diagonal(H)

    threshold = 2 * X_train.shape[1] / X_train.shape[0]

    X_outliers=[]
    for i in range (len(H_ii)):
        if H_ii[i] > threshold:
            X_outliers.append(i)
            
    return X_outliers, H_ii

X_outliers, H_ii = get_X_outliers(X_train_corr)
print(len(X_outliers))

In [18]:
def get_Y_outliers(X_train, model, H_ii):
    """Return the indexes of the outliers in the dataset with respect to Y"""

    residu = np.array(model.resid)
    SSE =  np.dot(residu.T,residu)

    Di_star = []
    #Compute the Di* for each entry
    for i in range (X_train.shape[0]):
        num = (X_train.shape[0] - X_train.shape[1] - 1)

        denom = (SSE * (1 - H_ii[i]) - residu[i]*residu[i] )
        Di = np.sqrt(num/ denom)*residu[i]
        Di_star.append(np.abs(Di))

    # Compute the threshold
    threshold = t.ppf(1-0.05/2, X_train.shape[0] - X_train.shape[1]-1)

    # Get the indexes of the outliers based on the threshold
    Y_outliers=[]
    for i in range (len(Di_star)):
        if (Di_star[i] > threshold):
            Y_outliers.append(i) 

    return Y_outliers

Y_outliers = get_Y_outliers(X_train_corr, model, H_ii)


#### Influential observations

In [19]:
def get_influencial_obs_DFFITS(model):
    """Get the indexes of the influencial observations based on DFFITS"""
    dffits = model.get_influence().dffits[0]

    # Compute the threshold
    threshold = 2 * np.sqrt(1/X_train_corr.shape[0] + 1/X_train_corr.shape[1])

    # Get the indexes of the influential observations based on the threshold
    return np.where(np.abs(dffits) > threshold)[0]

def get_influencial_obs_DFBETAS(model):
    """Get the indexes of the influencial observations based on DFBETAS"""
    dfbetas = model.get_influence().dfbetas

    # Compute the threshold
    dfbetas_treshold = 2 * np.sqrt(X_train_corr.shape[1] / X_train_corr.shape[0])

    # Get the indexes of the influential observations based on the threshold
    influential_obs_dfbetas = []
    for i in range(len(dfbetas)):
        for j in range(len(dfbetas[i])):
            if (np.abs(dfbetas[i][j]) > dfbetas_treshold):
                influential_obs_dfbetas.append(i)
                break

    return influential_obs_dfbetas


def get_influencial_obs_cooks(model):
    """Get the indexes of the influencial observations based on Cook's distance"""
    cooks_distance = model.get_influence().cooks_distance[0]

    n = X_train_corr.shape[0]
    p = X_train_corr.shape[1]
    alpha = 0.05

    #compute the threshold
    cooks_distance_treshold = f.ppf(1 - alpha, p, n - p)

    # Get the indexes of the influential observations based on the threshold
    influential_obs_cooks = np.where(cooks_distance > cooks_distance_treshold)
    return influential_obs_cooks[0]

influencial_obs_DFFITS = get_influencial_obs_DFFITS(model)
influencial_obs_DFBETAS = get_influencial_obs_DFBETAS(model)
influencial_obs_cooks = get_influencial_obs_cooks(model)

#### Multicollinearity

In [ ]:
def get_column_col(X_train):
    """Get the features that have a VIF > 10"""
    # Convert the DataFrame to a matrix
    X = X_train.drop(["const"], axis=1)
    X = X.values

    # Compute the VIF for each column
    vif = [variance_inflation_factor(X, i) for i in range(X.shape[1])]

    # Print the VIF for each column
    print(vif)

    # Get the indexes of the features that have a VIF > 10
    features_col = []
    for i in range(len(vif)):
        if vif[i] > 10:
            features_col.append(i)

    colum_name = X_train.drop(["const"], axis=1).columns[features_col]
    print(colum_name)

get_column_col(X_train_corr)

#### heteroskedasticty

In [21]:
def verify_hetero_white(model):
    """Verify the assumption of homoscedasticity with white test"""
    # Perform the Goldfeld-Quandt test
    _, pval, _, _= sm.stats.diagnostic.het_white(model.resid, model.model.exog)

    # Check the p-value
    if pval < 0.05:
        print("The assumption is violated.")
    else:
        print("The assumption is not violated.")

verify_hetero_white(model)

The assumption is violated.


In [22]:
def verify_hetero_gold(model):
    """Verify the assumption of homoscedasticity with goldfeld test"""
    # Perform the Goldfeld-Quandt test
    _, pval, _= sm.stats.diagnostic.het_goldfeldquandt(model.resid, model.model.exog)

    # Check the p-value
    if pval < 0.05:
        print("The assumption is violated.")
    else:
        print("The assumption is not violated.")

verify_hetero_gold(model)

The assumption is not violated.


### Normality of the residuals

In [23]:
def verifiy_normal(model, thresh=0.05):
    """Verify the assumption of normality of the residuals with Anderson-Darling test"""

    resid = model.resid

    # Performing the Anderson-Darling test on residuals
    p_value = sm.stats.diagnostic.normal_ad(resid)[1]

    # Reporting the normality of the residuals
    if p_value < thresh:
        print('Assumption not verified')
    else:
        print('Assumption verified')

    plt.subplots(figsize=(12, 6))
    plt.title('Distribution of Residuals')
    sns.distplot(resid)
    plt.show()



## Improvments

### Multicollinearity

In [24]:
# Drop the features selected to remove multicollinearity
X_train_corr = X_train_corr.drop(["review_palate"], axis=1)
X_test_corr = X_test_corr.drop(["review_palate"], axis=1)

### Remove influential observations

In [25]:
X_train_wo = X_train_corr.drop(X_train_corr.index[influencial_obs_DFBETAS])
y_train_wo = y_train.drop(y_train.index[influencial_obs_DFBETAS])


### Robust OLS

In [26]:
model = sm.OLS(y_train_wo, X_train_wo).fit(cov_type='HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         review_overall   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     1131.
Date:                Sun, 08 Jan 2023   Prob (F-statistic):               0.00
Time:                        00:07:09   Log-Likelihood:                 2001.3
No. Observations:                2494   AIC:                            -3951.
Df Residuals:                    2468   BIC:                            -3799.
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [ ]:
verifiy_normal(model)

## Test of significance

In [ ]:
# Get the pvalues and tvalues
print(model.pvalues)
print(model.tvalues)

## Linear combination

In [30]:
def lab_prediction(df):
    """Create linear combination of review_aroma, review_appearance, review_taste and review_overall coefficients"""
    return df["review_aroma"] * 0.0034 + df["review_appearance"] * 0.0323 + df["review_taste"] * 0.4243 + 3.7525
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = y_train, y=lab_prediction(X_train_corr) ,  # y values
    mode='markers',name = "prediction"  
))

fig.add_trace(go.Scatter(
        x=[2, 3, 5 ],  # x values
        y=[2, 3, 5 ],  # y values
        mode='lines'  
    ))

fig.show()

#print(compute_RMSE(y_train, lab_prediction(X_train_corr)))


## Coefficients equal to 0

In [31]:
def review_prediction(df , ynewpred):
    """Set the style coefficients to 0"""
    return ynewpred - df["Style_Bock"] * -0.0171 - df["Style_Brown Ale"] * 0.0134 - df["Style_IPA"] * 0.0031 - df["Style_dark"] * -0.0148  - df["Style_lager"]*-0.0022

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = y_train, y=lab_prediction(X_train_corr) ,  # y values
    mode='markers',name = "prediction"  
))

fig.add_trace(go.Scatter(
        x=[2, 3, 5 ],  # x values
        y=[2, 3, 5 ],  # y values
        mode='lines'  
    ))

fig.show()

#print(compute_RMSE(y_train, lab_prediction(X_train_corr)))

## Predictions


In [32]:
ynewpred = model.predict(X_test_corr)

In [33]:
def display_prediction(y_test, ynewpred):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=[2, 3, 5 ],  # x values
        y=[2, 3, 5 ],  # y values
        mode='lines'  
    ))
    fig.add_trace(go.Scatter(
        x=ynewpred , y=y_test,  # y values
        mode='markers'  
    ))


    fig.update_layout(
        xaxis=dict(
            title='prediction'
        ),
        yaxis=dict(
            title='y_test'
        )
    )
    fig.show()


In [34]:
display_prediction(y_test, ynewpred)

In [35]:
def compute_RMSE(y_test, ynewpred):
    # Calculate the RMSE of the predictions
    rmse = np.sqrt(mean_squared_error(y_test, ynewpred))
    print(rmse)

compute_RMSE(y_test, ynewpred)

0.13415565206317856


## Prediction interval

In [36]:
X_test_corr_copy = X_test_corr.copy()
y_test_copy = y_test.copy()
X_test_corr_copy["overall review"] = (y_test_copy)

#Sort the dataframe by overall review
X_test_corr_copy = X_test_corr_copy.sort_values(by='overall review', ascending=True)
y_test_copy=X_test_corr_copy.pop('overall review')

#Predict the values
ynewpred2 = model.predict(X_test_corr_copy)
predictions = model.get_prediction(X_test_corr_copy)

#print(predictions.summary_frame(alpha=0.05))


In [37]:
#Create the confidence interval
prediction_interval = predictions.conf_int(obs=True, alpha=0.05)
lower_bounds = [bounds[0] for bounds in prediction_interval]
upper_bounds = [bounds[1] for bounds in prediction_interval]

In [38]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=ynewpred2 ,  # y values
    mode='markers',name = "prediction" 
))
fig.add_trace(go.Scatter(
    y=lower_bounds ,  # y values
    mode='markers',name = "lower_bounds"
))
fig.add_trace(go.Scatter(
    y=upper_bounds,  # y values
    mode='markers',name = "upper_bounds"
))
fig.add_trace(go.Scatter(
    y=y_test_copy ,  # y values
    mode='markers',name = "real values"
))


fig.update_layout(
    yaxis=dict(
        title='Overall review'
    ),

)
fig.show()


In [39]:
#Compute the rate of prediction within the confidence interval
N=0
for i in range (len(y_test_copy)):

    if ((y_test_copy.iloc[i] > lower_bounds[i]) and (y_test_copy.iloc[i] < upper_bounds[i])):
        N=N+1
print(N/len(ynewpred2))


0.9265625
